# Flat Image Net - Evaluate Results

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from utils.nn_graph import simple_layer
from utils.nn_visualization import variable_summaries, img_summaries
from utils.data import init_model_logging
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

## Load data

In [ ]:
data = input_data.read_data_sets('/data/fashion/', one_hot=True)
class_id2class_name_mapping = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot'}

## Build Net Graph

In [ ]:
graph = tf.Graph()
with graph.as_default():
    with tf.name_scope('flat_image_net_inputs'):
        images = tf.placeholder(tf.float32, shape=[None, 784], name='images')
        labels = tf.placeholder(tf.float32, shape=[None, 10], name='labels')
        keep_dropout_prob = tf.placeholder(tf.float32, name='keep_dropout_prob')

    
    with tf.variable_scope('simple_layer_1'):
        raw_prediction = simple_layer(name='layer1', input_data=images, shape=[784, 64], activation='relu')
        
    with tf.variable_scope('simple_layer_2'):
        raw_prediction = simple_layer(name='layer2', input_data=raw_prediction, shape=[64, 10])
        raw_prediction = tf.nn.dropout(raw_prediction, keep_dropout_prob)
            
    with tf.name_scope('prediction'):
        prediction = tf.nn.softmax(raw_prediction)
    
    with tf.name_scope('loss'):
        cross_entropy_vector = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=prediction)
        loss = tf.reduce_mean(cross_entropy_vector)
        variable_summaries('loss_summary', cross_entropy_vector)

    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
        correct_prediction = tf.cast(correct_prediction, tf.float32)
        accuracy = tf.reduce_mean(correct_prediction)
        variable_summaries('accuracy_summary', correct_prediction)       
        
    with tf.name_scope('training'):
        train_step = tf.train.AdamOptimizer(0.001).minimize(loss)
            
    initialize_vars = tf.global_variables_initializer()
    merge_summaries = tf.summary.merge_all()

## Init Model Logging

In [ ]:
base_dir = '/saved_tensorboard_summaries/flat_image_net/'
logging_meta = init_model_logging(base_dir, 'experiment_final', graph=graph, remove_existing=False)
logging_meta

## Run Net

In [ ]:
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
model_path = logging_meta['model_path']+'-5000'
        
session = tf.Session(graph=graph, config=config)
logging_meta['saver'].restore(session, model_path)
_prediction, = session.run([prediction], feed_dict={images: data.validation.images, keep_dropout_prob: 1.0})

In [ ]:
session.close()

In [ ]:
_prediction

## Results evaluation

In [ ]:
from utils.results_evaluation import get_info_df
from utils.results_evaluation import plot_examples
from utils.results_evaluation import plot_coocurance_matrix

### Load reuslts to pandas df

In [ ]:
df = get_info_df(data.validation.labels, _prediction, class_id2class_name_mapping, images=data.validation.images)
df

In [ ]:
df.columns

### Precision

In [ ]:
def get_precision(df, class_name):
    true_positives = len(df[(df.label_class_name==class_name) & (df.predicted_class_name_top1==class_name)])
    positives = len(df[(df.predicted_class_name_top1==class_name)])
    if positives == 0:
        positives = 1
    return round(true_positives / positives * 100, 2)

In [ ]:
get_precision(df, 'T-shirt/top')

### Recall

In [ ]:
def get_recall(df, class_name):
    true_positives = len(df[(df.label_class_name==class_name) & (df.predicted_class_name_top1==class_name)])
    trues = len(df[(df.label_class_name==class_name)])
    if trues == 0:
        trues = 1
    return round(true_positives / trues * 100, 2)

In [ ]:
get_recall(df, 'T-shirt/top')

### Accuracy

In [ ]:
def get_accuracy(df):
    return round(float(np.mean((df.label_class_name == df.predicted_class_name_top1).astype(int))) * 100, 2)

In [ ]:
get_accuracy(df)

### Overall Recall and Precision

In [ ]:
def get_rec_prec(df, class_id2class_name_mapping):
    return pd.DataFrame(
        {
            "class_name": [class_name for class_name in class_id2class_name_mapping.values()],
            "recall": [get_recall(df, class_name) for class_name in class_id2class_name_mapping.values()],
            "precision": [get_precision(df, class_name) for class_name in class_id2class_name_mapping.values()]
        })

In [ ]:
get_rec_prec(df, class_id2class_name_mapping).sort_values(by='recall')

### False Positives

In [ ]:
def get_false_positives(df, label_class_name, predicted_class_name=None):
    if predicted_class_name is None:
        condition = (df['label_class_name']==label_class_name) & (df['predicted_class_name_top1']!=label_class_name)
    else:
        condition = (df['label_class_name']==label_class_name) & (df['predicted_class_name_top1']==predicted_class_name)
    return df[condition].sort_values(by='predicted_class_score_top1', ascending=False)

In [ ]:
fp = get_false_positives(df, 'Shirt', 'Coat')
fp

In [ ]:
plot_examples(fp)

### Coocurance matrix

In [ ]:
plot_coocurance_matrix(df, use_log=False, use_top3=False)

In [ ]:
plot_coocurance_matrix(df, use_log=True)